<a href="https://colab.research.google.com/github/nikshrimali/TSAI_END2_Phase1/blob/main/qna_BARTmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install faiss
!sudo apt-get install libomp-dev
!pip install transformers

     |████████████████████████████████| 4.7 MB 5.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (302 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Re

In [ ]:
def seed_all(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
# BERT imports

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline

from transformers import BertTokenizer, BertModel
from transformers import BartConfig, BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

import faiss

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader


# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
        print('Found GPU at: {}'.format(device_name))

else:
    print(f'Currently Running on {device}')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Currently Running on cuda


In [ ]:
rm -r '/content/sample_data'

In [ ]:
!rm -r '/content/END2_Capstone'

rm: cannot remove '/content/END2_Capstone': No such file or directory


In [ ]:
# Reading the data
# df_context = pd.read_csv('/content/drive/MyDrive/END2_CAPSTONE/context_groups.csv') 
# df_merged = pd.read_csv('/content/drive/MyDrive/END2_CAPSTONE/train.csv')
# model_op = torch.load('/content/drive/MyDrive/END2_CAPSTONE/np_encoded_context.pt')


df_context = pd.read_csv('/content/drive/MyDrive/END2_Capstone/context_groups.csv') 
df_merged = pd.read_csv('/content/drive/MyDrive/END2_Capstone/merged_data.csv')
model_op = torch.load('/content/drive/MyDrive/END2_Capstone/encoded_context.pt')

In [ ]:
# torch.save(model_op, '/content/drive/MyDrive/END2_CAPSTONE/np_encoded_context.pt')

In [ ]:
class GetEncodings:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')


    def encode(self, text, max_length=200):
        '''Tokenize data and get encoded embeddings from model'''
        tk = self.tokenizer(
                text,
                max_length=max_length,
                padding='max_length',
                return_attention_mask=True,
                add_special_tokens=True,
                truncation=True,
                return_tensors="pt"
            )
        model_op = self.model(**tk)['pooler_output']
        return model_op

In [ ]:
ge = GetEncodings()

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class SearchSimilar:
    def __init__(self, iterator = None, filename=None, embeddings=None, shape=None, device="cpu"):

        self.iterator = iterator
        
        if os.path.exists(filename) == True:

            print(f'Index file {filename}')
            self.index = faiss.read_index(filename)  # index2 is identical to index
        
        else:

            self.index = faiss.index_factory(shape, "Flat", faiss.METRIC_INNER_PRODUCT)
            faiss.normalize_L2(embeddings)
            self.index.add(embeddings)
            faiss.write_index(self.index, filename)
            print(f'Index written at {filename}')
 
        
        if device == "cuda":
            print('Now running on CUDA')
            self.index = faiss.index_cpu_to_all_gpus(self.index)

  
        print(f'Index trained - {self.index.is_trained}')


    def get_n_similar_vectors(self, text, n=2):
        '''Find the top n similar vector from text using cosine similarity FAISS'''
        text = text.cpu().detach().numpy()
        faiss.normalize_L2(text)
        distance, index = self.index.search(text, n)

        context = []

        for i in index.tolist()[0]:
            context.append(self.iterator[i])
        return context
    
    def get_n_dissimilar_vectors(self, text, n=2):
        '''Find the top n dissimilar vector from text using cosine similarity FAISS'''
        pass

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# install huggingface BART MOdel
class TextTokenizer(torch.utils.data.Dataset):
    def __init__(
        self,
        data:pd.DataFrame, 
        tokenizer:BartTokenizer,
        context_list:list,
        source_max_length: int=1024,
        target_max_length: int=30,
        device='cpu'
        ):

        self.tokenizer = tokenizer
        self.data = data
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length

        self.context_list = context_list

        self.ss = SearchSimilar(iterator = context_list, filename='index.bin', embeddings=model_op, shape=768, device=device)


    def __len__(self):
        return len(self.data)


    def __getitem__(self, index:int):
        data_row = self.data.iloc[index]

        context_index = data_row['group_id'].item()
        context = ge.encode(self.context_list[context_index])


        similar_contexts = self.ss.get_n_similar_vectors(context, 3)
        similar_contexts.insert(0, data_row['question'])

        combined_tokens = '</s></s>'.join(similar_contexts)

        source_encoding = tokenizer(
            combined_tokens,
            max_length=1024,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")

        
        target_encoding = tokenizer(
            data_row['answer'],
            max_length=self.target_max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
        

        # labels = target_encoding['input_ids']
        # decoder_input_ids = shift_tokens_right(labels, model.config.pad_token_id).flatten()
        # labels[labels[:, :] == self.model.config.pad_token_id] = -100
        # print(label)
    


        return dict(
            input_ids_source_merged = source_encoding['input_ids'].flatten(),
            attention_mask_source_merged = source_encoding['attention_mask'].flatten(),
            input_ids_target = target_encoding['input_ids'].flatten(),
            attention_mask_target = target_encoding['attention_mask'].flatten(),
        )


In [ ]:
text = TextTokenizer(data=df_merged, tokenizer=tokenizer, context_list=df_context['context'].values.tolist(), device=device)

Index written at index.bin
Index trained - True


In [ ]:
trainloader = torch.utils.data.DataLoader(text, batch_size=4)

In [ ]:
class BARTTrain(torch.nn.Module):
    def __init__(self):
        super(BARTTrain, self).__init__()
        self.model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
    
    def forward(self, encodings, mode):
        print('dvadfdsfds', mode)
        if mode == 'train':
            model_outputs = self.model(
                input_ids=encodings['input_ids_source_merged'].to(device), 
                attention_mask=encodings['attention_mask_source_merged'].to(device),
                decoder_input_ids=encodings['input_ids_target'].to(device),
                decoder_attention_mask=encodings['attention_mask_target'].to(device),
                labels=encodings['input_ids_target'].to(device))
        else:
            model_outputs = self.model.generate(encodings)     

        return model_outputs

In [ ]:
import glob
MODEL_STORE = '/content/drive/MyDrive/END2_CAPSTONE'
checkpoints = sorted(glob.glob(f'{MODEL_STORE}/checkpoint*-[0-9]*'))

In [ ]:
checkpoints

['/content/drive/MyDrive/END2_CAPSTONE/checkpoint-103']

In [ ]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

if len(checkpoints) > 0:
    global_step = int(checkpoints[0].split('-')[-1])
    ckpt_name = 'checkpoint-{}'.format(global_step)
    print("Loading model from checkpoint %s" % ckpt_name)
    
    PATH = f'{MODEL_STORE}/{ckpt_name}/checkpoint-{str(global_step)}.pt'
    loss_path = MODEL_STORE + '/'+ ckpt_name + '/training_loss.pt'

    model.load_state_dict(torch.load(PATH)) 
    train_loss_set_ckpt = torch.load(loss_path)
    train_loss_set = to_list(train_loss_set_ckpt)
    tr_loss = train_loss_set[-1]
else:
    global_step = 0
    train_loss_set = []
    tr_loss = 0.0
    model = BARTTrain().to(device)


Loading model from checkpoint checkpoint-103


In [ ]:
learning_rate = 5e-5
adam_epsilon=1e-8
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

In [ ]:

gc.collect()
torch.cuda.empty_cache()

In [ ]:

num_train_epochs = 10
fraction_per_epoch = 0.2
# global_step
seed_all()


print("***** Running training *****")
print("  Num examples = %d" % len(text))
print("  Num Epochs = %d" % num_train_epochs)
print("  Total optimization steps = %d" % (len(trainloader) // (num_train_epochs * fraction_per_epoch)))

model.zero_grad()
train_iterator = trange(num_train_epochs, desc="Epoch")

train_loss_set = []
loss = 0

for _ in train_iterator:
    print(loss)
    epoch_iterator = tqdm(iter(trainloader), desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      model.train()

      model_output = model(batch)
      loss = model_output[0]
      train_loss_set.append(loss)
      loss.backward()
      
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      tr_loss += loss
      optimizer.step()
      model.zero_grad()
      global_step += 1
      ckpt_name = 'checkpoint-{}'.format(global_step)

      if global_step % 109 == 0:
        print("Train loss: {}".format(tr_loss/global_step))
        output_dir = os.path.join(MODEL_STORE,'checkpoint-{}'.format(global_step))
        PATH = f'{output_dir}/{ckpt_name}.pt'
        loss_path = f'{output_dir}/training_loss.pt'


        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        torch.save(model_to_save.state_dict(), PATH)
        torch.save(torch.tensor(train_loss_set), loss_path)
        print("Saving model checkpoint to %s" % output_dir)


***** Running training *****
  Num examples = 50
  Num Epochs = 10
  Total optimization steps = 6


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

0


Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

Train loss: 0.014443718828260899
Saving model checkpoint to /content/drive/MyDrive/END2_CAPSTONE/checkpoint-109
tensor(0.2323, device='cuda:0', grad_fn=<NllLossBackward>)


...

Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

tensor(0.1756, device='cuda:0', grad_fn=<NllLossBackward>)


Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

Train loss: 0.11786329001188278
Saving model checkpoint to /content/drive/MyDrive/END2_CAPSTONE/checkpoint-218
tensor(0.2284, device='cuda:0', grad_fn=<NllLossBackward>)


Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
def inference(val_loader):
    print("***** Running evaluation *****")
    print("  Num examples = %d" % len(dataset))
    print("  Batch size = %d" % batch_size)
    all_results = []
    for batch in tqdm(val_loader, desc="Evaluating", miniters=100, mininterval=5.0):
        model.eval()
        with torch.no_grad():

            model_output = model(batch)
            loss = model_output[0]


In [ ]:

def load_model(checkpoint, model):

    MODEL_STORE = '/content/drive/MyDrive/END2_CAPSTONE'
    checkpoints = sorted(glob.glob(f'{MODEL_STORE}/{checkpoint}*-[0-9]*'))
    global_step = int(checkpoints[0].split('-')[-1])
    ckpt_name = '{}-{}'.format(checkpoint, global_step)
    print("Loading model from checkpoint %s" % ckpt_name)
    
    PATH = f'{MODEL_STORE}/{ckpt_name}/{checkpoint}-{str(global_step)}.pt'

    model.load_state_dict(torch.load(PATH))
    print("model successfully loaded %s" % ckpt_name)

    return model


def get_outputs(inputs):
    
    tokens = tokenizer.encode('I am nikhil',  return_tensors='pt')
    model_1 = BARTTrain().to(device)
    # model_2 = BARTTrain().to(device)
    model_1 = load_model('checkpoint', model_1)
    model_2 = load_model('qna_checkpoint', model_1)

    output = model_1(tokens.to(device), mode = 'eval')
    output = tokenizer.decode(output[0])
    print(output)
    return output



In [ ]:
get_outputs('who invented pytorch')

Loading model from checkpoint checkpoint-103
model successfully loaded checkpoint-103
dvadfdsfds eval
</s><s>III II I I II, I I, I, and I</s>


'</s><s>III II I I II, I I, I, and I</s>'

In [ ]:
# model('I am nikhil')
a = tokenizer.encode('I am nikhil',  return_tensors='pt')
# model(tokenizer.encode('I am nikhil shrimali'))

In [ ]:
model = BARTTrain().to(device)

In [ ]:
b = model(encodings=a.to(device), mode='eval')

dvadfdsfds eval


In [ ]:
b

tensor([[    2,     0,   100,   524,   295, 18439,   718,     2]],
       device='cuda:0')

In [ ]:
tokenizer.decode(b[0])

'</s><s>I am nikhil</s>'